<a href="https://colab.research.google.com/github/Luensmann/Bachelorarbeit/blob/main/Pre%20Processing/split_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from transformers import AutoTokenizer, BertForTokenClassification, AutoModelForTokenClassification
from datasets import load_dataset
from datasets import Dataset, DatasetDict

In [ ]:
dataset = load_dataset("Brizape/SETH_0404")
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating test split:   0%|          | 0/126 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/504 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/pc/.cache/huggingface/datasets/Brizape___parquet/Brizape--SETH_0404-783d4790b0e24b4b/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    test: Dataset({
        features: ['id', 'texts', 'tokens', 'ner_tags'],
        num_rows: 126
    })
    train: Dataset({
        features: ['id', 'texts', 'tokens', 'ner_tags'],
        num_rows: 504
    })
})

In [ ]:
for data_index, data in enumerate(dataset['train']['tokens'][:3]):
    print("#"*(30), data_index, len(data), ("#"*(30)))
    j = 0
    for i, token in enumerate(data):
        j = j+1
        if(token == "\n\n" or token == "\n"):
            j = 0
        if(j > 400):
            print("#~"*(60))
        print(j, i, repr(token))

In [ ]:
print(repr(new_data['test']['texts'][:5]))
print(repr(dataset['train']['texts'][0]))

In [ ]:
# Function to split rows
def split_rows(dataset):
    new_rows = []

    for row in dataset:
        tokens = row['tokens']
        ner_tags = row['ner_tags']
        
        split_indices = [i for i, t in enumerate(tokens) if t == '\n\n' or t == '\n']

        prev_idx = 0
        for idx in split_indices:
            new_row = {k: row[k] for k in row if k not in ['tokens', 'ner_tags', 'texts']}
            new_row['tokens'] = tokens[prev_idx:idx]
            new_row['ner_tags'] = ner_tags[prev_idx:idx]
            new_row['texts'] = ' '.join(new_row['tokens'])
            new_rows.append(new_row)
            prev_idx = idx + 1

        new_row = {k: row[k] for k in row if k not in ['tokens', 'ner_tags', 'texts']}
        new_row['tokens'] = tokens[prev_idx:]
        new_row['ner_tags'] = ner_tags[prev_idx:]
        new_row['texts'] = ' '.join(new_row['tokens'])
        new_rows.append(new_row)

    return new_rows

In [ ]:
# Splitting rows and creating a new DatasetDict
new_data = DatasetDict()
for split_name in dataset.keys():
    split_data = split_rows(dataset[split_name])
    new_data[split_name] = Dataset.from_dict({k: [row[k] for row in split_data] for k in split_data[0]})

print(new_data)

DatasetDict({
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts'],
        num_rows: 126
    })
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'texts'],
        num_rows: 504
    })
})


In [ ]:
new_data.push_to_hub("Brizape/SETH_split_0404")

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
for data_index, (data, tags) in enumerate(zip(new_data['test']['token'][:3], new_data['test']['ner_tags'][:3])):
    print("#"*(30), data_index, len(data), ("#"*(30)))
    j = 0
    for i, (token, tag) in enumerate(zip(data, tags)):
        j = j+1
        if(token == "\n\n" or token == "\n"):
            j = 0
        if(j > 400):
            print("#~"*(60))
        print(j, i, repr(token), tag)